In [13]:
from pyspark.sql import SparkSession
import collections


In [14]:
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()



23/01/15 03:47:27 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [21]:
df = spark.read.json("../sample-data/Spark_DataFrames/people.json")
df.head(5)
df.printSchema()
#this infered the schema from automatically
#May default to string sometimes when its not able to infer the schema based on the values privided

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [24]:
df.describe()

DataFrame[summary: string, age: string, name: string]

In [26]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [16]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [27]:
data_schema = [StructField('age', IntegerType(), True),
               StructField('name', StringType(), True)]
#name , type, can field be null or not (Trye for yes)

In [32]:
final_struct  = StructType(fields = data_schema)
type(final_struct)

pyspark.sql.types.StructType

In [62]:
#read using our defined schema
lines = spark.read.json("sample-data/Spark_DataFrames/people.json", final_struct)


In [63]:
lines.head()

Row(age=None, name='Michael')

In [39]:
#printiing our schema
lines.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [41]:
lines.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [45]:
lines['age']
type(lines['age'])
#this returns a column, you have to use select if you wish to select a column

pyspark.sql.column.Column

In [47]:
lines.select('age') 
#returns a dataframe instead of a column

DataFrame[age: int]

In [50]:
lines.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [56]:
lines.head(2)[0]
#returns pyspark.sql.types.Row

Row(age=None, name='Michael')

In [57]:
#select multiple columns
lines.select('age', 'name').show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [73]:
#create a new dataframe with a new column
df = spark.read.json("sample-data/Spark_DataFrames/people.json", final_struct)
df.withColumn('doubleAge', df['age']*2).show()

+----+-------+---------+
| age|   name|doubleAge|
+----+-------+---------+
|null|Michael|     null|
|  30|   Andy|       60|
|  19| Justin|       38|
+----+-------+---------+



In [74]:
df.withColumnRenamed('age', 'my_new_age').show()
#rename existing column

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



In [76]:
#using pure sql to select a dataframe
df.createOrReplaceTempView('people_temp_view') #register this df as a vew


In [78]:
results = spark.sql("SELECT * FROM people_temp_view WHERE age = 30")

In [80]:
results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

